## Advanced World Space Demo
advanced world space features including mesh clipping, orientation options, and multiplanar viewing with meshes

See https://niivue.com/demos/features/worldspace2.html for mirror.

In [ ]:
from pathlib import Path

from ipyniivue import download_dataset

BASE_API_URL = "https://niivue.com/demos/images/"
DATA_FOLDER = Path("images")

# Download data for example
download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "mni152.nii.gz",
        "BrainMesh_ICBM152.lh.mz3",
        "dpsv.trx",
    ],
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

from ipyniivue import DragMode, NiiVue, ShowRender, SliceType

## Create NiiVue Instance

nv = NiiVue(
    height=400,
    drag_and_drop_enabled=True,
    back_color=(0.3, 0.2, 0.4, 1.0),
    show_3d_crosshair=True,
)

nv.set_slice_mm(True)
nv.set_clip_plane(-0.1, 270, 0)
nv.set_render_azimuth_elevation(120, 10)
nv.set_high_resolution_capable(True)

nv.load_volumes(
    [
        {
            "path": DATA_FOLDER / "mni152.nii.gz",
            "colormap": "gray",
            "opacity": 1.0,
        }
    ]
)

nv.load_meshes(
    [
        {
            "path": DATA_FOLDER / "BrainMesh_ICBM152.lh.mz3",
            "rgba255": [200, 162, 255, 255],
        },
        {
            "path": DATA_FOLDER / "dpsv.trx",
            "rgba255": [255, 255, 255, 255],
        },
    ]
)

nv.opts.slice_type = SliceType.MULTIPLANAR

## Create Interactive Controls

# mesh clipping slider
mesh_clipping_slider = widgets.IntSlider(
    value=11,
    min=0,
    max=11,
    description="Mesh clipping:",
    continuous_update=True,
    style={"description_width": "initial"},
)

# checkboxes
radiological_checkbox = widgets.Checkbox(value=False, description="Radiological")

corner_text_checkbox = widgets.Checkbox(value=False, description="Corner text")

world_space_checkbox = widgets.Checkbox(value=True, description="World space")

colorbar_checkbox = widgets.Checkbox(value=False, description="Colorbar")

cube_checkbox = widgets.Checkbox(value=False, description="Cube")

pad_checkbox = widgets.Checkbox(value=False, description="Pad")

high_dpi_checkbox = widgets.Checkbox(value=True, description="HighDPI")

force_render_checkbox = widgets.Checkbox(value=False, description="Force Render")

# drag mode dropdown
drag_mode_dropdown = widgets.Dropdown(
    options=[
        ("contrast", DragMode.CONTRAST),
        ("measurement", DragMode.MEASUREMENT),
        ("pan/zoom", DragMode.PAN),
        ("none", DragMode.NONE),
    ],
    value=DragMode.CONTRAST,
    description="Drag mode:",
    style={"description_width": "initial"},
)

## Setup Event Handlers


def on_mesh_clipping_change(change):
    """Handle mesh clipping slider changes."""
    dx = float(change["new"])
    if dx > 10:
        dx = float("inf")
    nv.set_mesh_thickness_on_2d(dx)


def on_radiological_change(change):
    """Handle radiological convention checkbox changes."""
    nv.set_radiological_convention(change["new"])


def on_corner_text_change(change):
    """Handle corner text checkbox changes."""
    nv.set_corner_orientation_text(change["new"])


def on_world_space_change(change):
    """Handle world space checkbox changes."""
    nv.set_slice_mm(change["new"])


def on_colorbar_change(change):
    """Handle colorbar checkbox changes."""
    nv.opts.is_colorbar = change["new"]


def on_cube_change(change):
    """Handle orientation cube checkbox changes."""
    nv.opts.is_orient_cube = change["new"]


def on_pad_change(change):
    """Handle multiplanar padding checkbox changes."""
    pad = 5 if change["new"] else 0
    nv.set_multiplanar_pad_pixels(pad)


def on_high_dpi_change(change):
    """Handle high DPI checkbox changes."""
    nv.set_high_resolution_capable(change["new"])


def on_force_render_change(change):
    """Handle force render checkbox changes."""
    if change["new"]:
        nv.opts.multiplanar_show_render = ShowRender.ALWAYS
    else:
        nv.opts.multiplanar_show_render = ShowRender.AUTO


def on_drag_mode_change(change):
    """Handle drag mode dropdown changes."""
    nv.opts.drag_mode = change["new"]


## Attach event handlers

mesh_clipping_slider.observe(on_mesh_clipping_change, names="value")
radiological_checkbox.observe(on_radiological_change, names="value")
corner_text_checkbox.observe(on_corner_text_change, names="value")
world_space_checkbox.observe(on_world_space_change, names="value")
colorbar_checkbox.observe(on_colorbar_change, names="value")
cube_checkbox.observe(on_cube_change, names="value")
pad_checkbox.observe(on_pad_change, names="value")
high_dpi_checkbox.observe(on_high_dpi_change, names="value")
force_render_checkbox.observe(on_force_render_change, names="value")
drag_mode_dropdown.observe(on_drag_mode_change, names="value")

## Initialize mesh clipping

on_mesh_clipping_change({"new": 11})  # sets to infinity cause >10

## Display All

controls_row1 = widgets.HBox([mesh_clipping_slider, drag_mode_dropdown])

controls_row2 = widgets.HBox(
    [
        radiological_checkbox,
        corner_text_checkbox,
        world_space_checkbox,
        colorbar_checkbox,
    ]
)

controls_row3 = widgets.HBox(
    [cube_checkbox, pad_checkbox, high_dpi_checkbox, force_render_checkbox]
)

controls = widgets.VBox([controls_row1, controls_row2, controls_row3])

display(widgets.VBox([controls, nv]))